In [54]:
import numpy as np
import pandas as pd

#Carga de datos
progNueva = pd.read_csv('progNueva.csv')
progTotal = pd.read_csv('progTotal.csv')

#prog = pd.read_csv('progNueva.csv')
progNueva['fechaVenta'] = pd.to_datetime(progNueva['fechaVenta'])
progTotal['fechaVenta'] = pd.to_datetime(progTotal['fechaVenta'])
progTotal['fechaProg'] = pd.to_datetime(progTotal['fechaProg'])

progNueva['fechaProg'] = progNueva['fechaVenta'] + pd.to_timedelta((progNueva['Semana_prog']-1), unit='w')
progTotal

,Nombre_Clinica,Paciente,dni,ttmto_homol,Cantidad,Cant_prod,Cant_pend,fechaVenta,Semana_prog,fechaProg
0,Autopista 104A Bogota,EVELYN DAZA,111,MENSUALIDAD ORTODONCIA METALICA,9,0,9,2018-01-02,6,2018-01-02
1,Autopista 104A Bogota,EVELYN DAZA,111,RETENEDOR ESSIX,1,0,1,2018-01-02,36,2018-01-02
2,Autopista 104A Bogota,EVELYN DAZA,111,RETENEDOR FIJO,2,0,2,2018-01-02,36,2018-01-02
3,Autopista 104A Bogota,EVELYN DAZA,111,RETIRO BRACKETS ARCADA,2,0,2,2018-01-02,1,2018-01-02
4,Autopista 104A Bogota,PEDRO GOMEZ,222,DETARTRAJE POR ARCADA,2,0,2,2018-01-03,1,2018-01-03
5,Autopista 104A Bogota,PEDRO GOMEZ,222,PROFILAXIS,2,0,2,2018-01-03,1,2018-01-03


In [55]:
# extraer coincidencias entre las programaciones, sumar Q´s, sumar semana_prog, conservar fechaVenta original, semana_prog, fechaProg
innerProg = pd.merge(progTotal, progNueva, how='inner', left_on=['Nombre_Clinica','dni','ttmto_homol'], right_on=['Nombre_Clinica','dni','ttmto_homol'])
innerProg['Cantidad'] = innerProg['Cantidad_x'] + innerProg['Cantidad_y']
innerProg['Cant_prod'] = innerProg['Cant_prod_x'] + innerProg['Cant_prod_y']
innerProg['Cant_pend'] = innerProg['Cant_pend_x'] + innerProg['Cant_pend_y']
innerProg = innerProg[['Nombre_Clinica','Paciente_x','dni','ttmto_homol','Cantidad','Cant_prod','Cant_pend','fechaVenta_x','Semana_prog_x','fechaProg_x']]
innerProg = innerProg.rename(index=str, columns={"Paciente_x":"Paciente","fechaVenta_x":"fechaVenta","Semana_prog_x":"Semana_prog",'fechaProg_x':'fechaProg'})
innerProg

,Nombre_Clinica,Paciente,dni,ttmto_homol,Cantidad,Cant_prod,Cant_pend,fechaVenta,Semana_prog,fechaProg
0,Autopista 104A Bogota,EVELYN DAZA,111,MENSUALIDAD ORTODONCIA METALICA,12,0,12,2018-01-02,6,2018-01-02


In [56]:
leftProg = pd.merge(progTotal, progNueva, how='left', left_on=['Nombre_Clinica','dni','ttmto_homol'], right_on=['Nombre_Clinica','dni','ttmto_homol'])
leftProg = leftProg[leftProg['Paciente_y'].isnull()]
leftProg = leftProg[['Nombre_Clinica','Paciente_x','dni','ttmto_homol','Cantidad_x','Cant_prod_x','Cant_pend_x','fechaVenta_x','Semana_prog_x','fechaProg_x']]
leftProg = leftProg.rename(index=str, columns={"Paciente_x":"Paciente","fechaVenta_x":"fechaVenta","Semana_prog_x":"Semana_prog",'fechaProg_x':'fechaProg','Cantidad_x':'Cantidad','Cant_prod_x':'Cant_prod','Cant_pend_x':'Cant_pend'})
leftProg

,Nombre_Clinica,Paciente,dni,ttmto_homol,Cantidad,Cant_prod,Cant_pend,fechaVenta,Semana_prog,fechaProg
1,Autopista 104A Bogota,EVELYN DAZA,111,RETENEDOR ESSIX,1,0,1,2018-01-02,36,2018-01-02
2,Autopista 104A Bogota,EVELYN DAZA,111,RETENEDOR FIJO,2,0,2,2018-01-02,36,2018-01-02
3,Autopista 104A Bogota,EVELYN DAZA,111,RETIRO BRACKETS ARCADA,2,0,2,2018-01-02,1,2018-01-02
4,Autopista 104A Bogota,PEDRO GOMEZ,222,DETARTRAJE POR ARCADA,2,0,2,2018-01-03,1,2018-01-03
5,Autopista 104A Bogota,PEDRO GOMEZ,222,PROFILAXIS,2,0,2,2018-01-03,1,2018-01-03


In [57]:
progTotal = leftProg.append(innerProg)

progTotal['PQR'] = 0

print("Suma de Programaciones:")
progTotal

Suma de Programaciones:


,Nombre_Clinica,Paciente,dni,ttmto_homol,Cantidad,Cant_prod,Cant_pend,fechaVenta,Semana_prog,fechaProg,PQR
1,Autopista 104A Bogota,EVELYN DAZA,111,RETENEDOR ESSIX,1,0,1,2018-01-02,36,2018-01-02,0
2,Autopista 104A Bogota,EVELYN DAZA,111,RETENEDOR FIJO,2,0,2,2018-01-02,36,2018-01-02,0
3,Autopista 104A Bogota,EVELYN DAZA,111,RETIRO BRACKETS ARCADA,2,0,2,2018-01-02,1,2018-01-02,0
4,Autopista 104A Bogota,PEDRO GOMEZ,222,DETARTRAJE POR ARCADA,2,0,2,2018-01-03,1,2018-01-03,0
5,Autopista 104A Bogota,PEDRO GOMEZ,222,PROFILAXIS,2,0,2,2018-01-03,1,2018-01-03,0
0,Autopista 104A Bogota,EVELYN DAZA,111,MENSUALIDAD ORTODONCIA METALICA,12,0,12,2018-01-02,6,2018-01-02,0
